# Kapitola 35: MNIST - Naucte AI cist rucne psane cislice

V teto kapitole postavime a natrénujeme skutecnou neuronovou sit v PyTorchu! Naucime AI rozpoznavat rucne psane cislice pomoci slavneho datasetu **MNIST**.

## Co se naucime:
- Jak pracovat s datasetem MNIST
- Jak vytvorit neuronovou sit v PyTorchu
- Trenovaci smycka v praxi
- Jak vyhodnotit presnost modelu
- Vizualizace predikci

---

## 1. Instalace a import knihoven

In [ ]:
# Instalace PyTorchu (pro Google Colab)
!pip install torch torchvision matplotlib

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Kontrola verze PyTorchu
print(f"PyTorch verze: {torch.__version__}")
print(f"Torchvision verze: {torchvision.__version__}")

# Kontrola GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Pouzivame zarizeni: {device}")

---

## 2. Co je MNIST?

**MNIST** (Modified National Institute of Standards and Technology) je databaze rucne psanych cislic:

| Vlastnost | Hodnota |
|-----------|--------|
| Pocet trenovacich obrazku | 60,000 |
| Pocet testovacich obrazku | 10,000 |
| Velikost obrazku | 28x28 pixelu |
| Pocet barev | 1 (sedotonovy) |
| Pocet trid | 10 (cislice 0-9) |

MNIST je "Hello World" hlubokého uceni - je idealni pro zacatecniky!

---

## 3. Nacteni datasetu MNIST

In [ ]:
# Hyperparametry
input_size = 784      # 28x28 = 784 pixelu
hidden_size = 128     # Pocet neuronu ve skryte vrstve
num_classes = 10      # 10 cislic (0-9)
num_epochs = 5        # Pocet pruchodu celym datasetem
batch_size = 100      # Pocet obrazku v jedne davce
learning_rate = 0.001 # Rychlost uceni

print("Hyperparametry nastaveny:")
print(f"  - Vstup: {input_size} neuronu ({28}x{28} pixelu)")
print(f"  - Skryta vrstva: {hidden_size} neuronu")
print(f"  - Vystup: {num_classes} trid")
print(f"  - Pocet epoch: {num_epochs}")
print(f"  - Velikost davky: {batch_size}")
print(f"  - Rychlost uceni: {learning_rate}")

In [ ]:
# Nacteni MNIST datasetu
# transforms.ToTensor() prevede obrazky na tensory s hodnotami 0-1

train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor()
)

print(f"Trenovaci obrazky: {len(train_dataset)}")
print(f"Testovaci obrazky: {len(test_dataset)}")

In [ ]:
# DataLoader - efektivni nacitani dat v davkach
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True  # Zamichani pro lepsi uceni
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

print(f"Pocet trenovacich davek: {len(train_loader)}")
print(f"Pocet testovacich davek: {len(test_loader)}")

---

## 4. Vizualizace dat

Podivejme se, jak obrazky vypadaji:

In [ ]:
# Nacteni jedne davky pro vizualizaci
examples = iter(test_loader)
example_data, example_targets = next(examples)

print(f"Tvar dat: {example_data.shape}")
print(f"  - batch_size: {example_data.shape[0]}")
print(f"  - kanaly: {example_data.shape[1]}")
print(f"  - vyska: {example_data.shape[2]}")
print(f"  - sirka: {example_data.shape[3]}")
print(f"Stitky: {example_targets[:10]}")

In [ ]:
# Zobrazeni prvnich 15 obrazku
fig, axes = plt.subplots(3, 5, figsize=(12, 8))

for i, ax in enumerate(axes.flat):
    ax.imshow(example_data[i][0], cmap='gray')
    ax.set_title(f'Cislice: {example_targets[i].item()}', fontsize=12)
    ax.axis('off')

plt.suptitle('Ukazka cislic z MNIST datasetu', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Detailni pohled na jeden obrazek
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Obrazek
axes[0].imshow(example_data[0][0], cmap='gray')
axes[0].set_title(f'Cislice: {example_targets[0].item()}', fontsize=14)
axes[0].axis('off')

# Hodnoty pixelu
im = axes[1].imshow(example_data[0][0], cmap='viridis')
axes[1].set_title('Hodnoty pixelu (0-1)', fontsize=14)
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

print(f"Min hodnota pixelu: {example_data[0][0].min():.4f}")
print(f"Max hodnota pixelu: {example_data[0][0].max():.4f}")

---

## 5. Definice neuronove site

Vytvorime jednoduchou sit se dvema vrstvami:

```
Vstup (784) -> Skryta vrstva (128) -> ReLU -> Vystup (10)
```

In [ ]:
# Definice architektury neuronove site
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # Prvni vrstva: vstup -> skryta
        self.layer1 = nn.Linear(input_size, hidden_size)
        # Aktivacni funkce
        self.relu = nn.ReLU()
        # Druha vrstva: skryta -> vystup
        self.layer2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Dopredny pruchod
        out = self.layer1(x)   # Prvni linearna transformace
        out = self.relu(out)    # Nelinearita (ReLU)
        out = self.layer2(out)  # Druha linearna transformace
        # Softmax se aplikuje v loss funkci (CrossEntropyLoss)
        return out

# Vytvoreni modelu
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

print("Architektura site:")
print(model)
print()

# Pocet parametru
total_params = sum(p.numel() for p in model.parameters())
print(f"Celkovy pocet parametru: {total_params:,}")

In [ ]:
# Vizualizace architektury
fig, ax = plt.subplots(figsize=(14, 6))

# Vrstvy
vrstvy = [
    ('Vstup\n784 neuronu\n(28x28 pixelu)', 784, 'lightblue'),
    ('Skryta vrstva\n128 neuronu\n+ ReLU', 128, 'lightgreen'),
    ('Vystup\n10 neuronu\n(cislice 0-9)', 10, 'lightsalmon')
]

for i, (nazev, pocet, barva) in enumerate(vrstvy):
    x = i * 4
    height = pocet / 100 if pocet > 50 else 1
    rect = plt.Rectangle((x, 2-height/2), 2.5, height, 
                         facecolor=barva, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(x + 1.25, 4, nazev, ha='center', va='bottom', fontsize=10)
    
    if i < len(vrstvy) - 1:
        ax.annotate('', xy=(x + 3.5, 2), xytext=(x + 2.5, 2),
                   arrowprops=dict(arrowstyle='->', color='gray', lw=2))

ax.set_xlim(-0.5, 10)
ax.set_ylim(-1, 6)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Architektura neuronove site pro MNIST', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---

## 6. Ztratova funkce a optimizer

In [ ]:
# Ztratova funkce - CrossEntropyLoss pro vicetridni klasifikaci
criterion = nn.CrossEntropyLoss()

# Optimizer - Adam (adaptivni rychlost uceni)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Ztratova funkce: CrossEntropyLoss")
print("  - Kombinuje Softmax + NLL Loss")
print("  - Idealni pro vicetridni klasifikaci")
print()
print("Optimizer: Adam")
print(f"  - Learning rate: {learning_rate}")

---

## 7. Trenovaci smycka

Toto je srdce uceni - model se postupne zlepsuje:

In [ ]:
# Trenovaci smycka
print("=" * 60)
print("ZAHAJUJI TRENINK")
print("=" * 60)

n_total_steps = len(train_loader)
loss_history = []  # Pro vizualizaci

for epoch in range(num_epochs):
    epoch_loss = 0
    
    for i, (images, labels) in enumerate(train_loader):
        # 1. PRIPRAVA DAT
        # Zplosteni obrazku: [100, 1, 28, 28] -> [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # 2. DOPREDNY PRUCHOD (Forward Pass)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 3. ZPETNY PRUCHOD (Backward Pass)
        optimizer.zero_grad()  # Vynulovani gradientu
        loss.backward()        # Vypocet gradientu
        
        # 4. AKTUALIZACE VAH
        optimizer.step()       # Krok optimalizace
        
        epoch_loss += loss.item()
        
        # Zobrazeni prubeznych vysledku
        if (i+1) % 100 == 0:
            print(f'Epocha [{epoch+1}/{num_epochs}], '
                  f'Krok [{i+1}/{n_total_steps}], '
                  f'Ztrata: {loss.item():.4f}')
    
    # Prumerna ztrata za epochu
    avg_loss = epoch_loss / n_total_steps
    loss_history.append(avg_loss)
    print(f'>>> Epocha {epoch+1} dokoncena, prumerna ztrata: {avg_loss:.4f}')
    print()

print("=" * 60)
print("TRENINK DOKONCEN!")
print("=" * 60)

In [ ]:
# Vizualizace trenovani
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), loss_history, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Epocha', fontsize=12)
plt.ylabel('Prumerna ztrata', fontsize=12)
plt.title('Ucici krivka - Ztrata behem treninku', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(range(1, num_epochs + 1))
plt.tight_layout()
plt.show()

print("Ztrata klesa = model se uci!")

---

## 8. Testovani a vyhodnoceni modelu

Jak si model vede na datech, ktera nikdy nevidel?

In [ ]:
# Testovani modelu
model.eval()  # Prepnuti do vyhodnocovaciho rezimu

with torch.no_grad():  # Nepocitame gradienty
    n_correct = 0
    n_samples = 0
    
    # Presnost pro kazdou tridu
    n_class_correct = [0] * 10
    n_class_samples = [0] * 10
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Predikce
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # Pro kazdou tridu
        for i in range(len(labels)):
            label = labels[i].item()
            pred = predicted[i].item()
            n_class_samples[label] += 1
            if label == pred:
                n_class_correct[label] += 1

# Celkova presnost
accuracy = 100.0 * n_correct / n_samples
print("=" * 60)
print(f"CELKOVA PRESNOST NA TESTOVACICH DATECH: {accuracy:.2f}%")
print("=" * 60)
print()

# Presnost pro jednotlive cislice
print("Presnost pro kazdou cislici:")
for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    bar = '█' * int(acc / 5)
    print(f"  Cislice {i}: {bar:20s} {acc:.1f}%")

---

## 9. Vizualizace predikci

Podivejme se, jak model predikuje:

In [ ]:
# Nacteni testovacich dat
examples = iter(test_loader)
example_data, example_targets = next(examples)

# Predikce
with torch.no_grad():
    example_data_flat = example_data.reshape(-1, 28*28).to(device)
    outputs = model(example_data_flat)
    _, predictions = torch.max(outputs, 1)

# Zobrazeni 15 predikci
fig, axes = plt.subplots(3, 5, figsize=(14, 9))

for i, ax in enumerate(axes.flat):
    ax.imshow(example_data[i][0].cpu(), cmap='gray')
    pred = predictions[i].item()
    true = example_targets[i].item()
    
    # Spravna predikce = zelena, chybna = cervena
    color = 'green' if pred == true else 'red'
    ax.set_title(f'Predikce: {pred}\nSkutecnost: {true}', 
                fontsize=11, color=color)
    ax.axis('off')

plt.suptitle('Predikce modelu na testovacich datech\n(zelena = spravne, cervena = chyba)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Vizualizace pravdepodobnosti pro jednu predikci
idx = 0  # Index obrazku

with torch.no_grad():
    single_image = example_data[idx:idx+1].reshape(-1, 28*28).to(device)
    output = model(single_image)
    probabilities = torch.softmax(output, dim=1)[0].cpu().numpy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Obrazek
axes[0].imshow(example_data[idx][0], cmap='gray')
axes[0].set_title(f'Skutecna cislice: {example_targets[idx].item()}', fontsize=14)
axes[0].axis('off')

# Pravdepodobnosti
bars = axes[1].bar(range(10), probabilities * 100, color='steelblue')
# Zvyrazneni predikce
predicted_idx = np.argmax(probabilities)
bars[predicted_idx].set_color('green')

axes[1].set_xlabel('Cislice', fontsize=12)
axes[1].set_ylabel('Pravdepodobnost (%)', fontsize=12)
axes[1].set_title(f'Predikce modelu: {predicted_idx}', fontsize=14)
axes[1].set_xticks(range(10))
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Pravdepodobnosti pro kazdou cislici:")
for i, prob in enumerate(probabilities):
    print(f"  {i}: {prob*100:.2f}%")

---

## 10. Ulozeni a nacteni modelu

In [ ]:
# Ulozeni modelu
torch.save(model.state_dict(), 'mnist_model.pth')
print("Model ulozen do: mnist_model.pth")

# Nacteni modelu (pro pozdejsi pouziti)
# loaded_model = NeuralNet(input_size, hidden_size, num_classes)
# loaded_model.load_state_dict(torch.load('mnist_model.pth'))
# loaded_model.eval()

---

## 11. Mini-kviz

In [ ]:
# Mini-kviz
print("=" * 60)
print("KVIZ: MNIST A NEURONOVE SITE V PYTORCH")
print("=" * 60)
print()

otazky = [
    {
        "otazka": "1. Kolik obrazku obsahuje trenovaci cast MNIST?",
        "moznosti": ["a) 10,000", "b) 28,000", "c) 60,000", "d) 100,000"],
        "spravna": "c",
        "vysvetleni": "MNIST obsahuje 60,000 trenovacich a 10,000 testovacich obrazku."
    },
    {
        "otazka": "2. Jaka je velikost jednoho MNIST obrazku?",
        "moznosti": ["a) 16x16", "b) 28x28", "c) 32x32", "d) 64x64"],
        "spravna": "b",
        "vysvetleni": "MNIST obrazky maji velikost 28x28 pixelu = 784 hodnot."
    },
    {
        "otazka": "3. Co dela funkce model.eval()?",
        "moznosti": [
            "a) Spusti trenink",
            "b) Prepne model do vyhodnocovaciho rezimu",
            "c) Smaze model",
            "d) Ulozi model"
        ],
        "spravna": "b",
        "vysvetleni": "model.eval() vypne dropout a batch normalization pro inference."
    },
    {
        "otazka": "4. Proc pouzivame CrossEntropyLoss?",
        "moznosti": [
            "a) Pro regresi",
            "b) Pro binarni klasifikaci",
            "c) Pro vicetridni klasifikaci",
            "d) Pro segmentaci"
        ],
        "spravna": "c",
        "vysvetleni": "CrossEntropyLoss je idealni pro vicetridni klasifikaci (10 cislic)."
    },
    {
        "otazka": "5. Co je 'epocha' v treninku?",
        "moznosti": [
            "a) Jeden krok optimalizace",
            "b) Jeden pruchod celym datasetem",
            "c) Jedna davka dat",
            "d) Jeden neuron"
        ],
        "spravna": "b",
        "vysvetleni": "Epocha = jeden kompletni pruchod vsemi trenovacimi daty."
    }
]

for q in otazky:
    print(q["otazka"])
    for m in q["moznosti"]:
        print(f"   {m}")
    print(f"   Spravna odpoved: {q['spravna']}")
    print(f"   > {q['vysvetleni']}")
    print()

---

## 12. Shrnuti kapitoly

| Krok | Popis |
|------|-------|
| **1. Data** | Nacteni MNIST pomoci torchvision |
| **2. DataLoader** | Efektivni nacitani dat v davkach |
| **3. Model** | nn.Module s Linear vrstvami a ReLU |
| **4. Loss** | CrossEntropyLoss pro klasifikaci |
| **5. Optimizer** | Adam s learning rate |
| **6. Trenink** | Forward -> Loss -> Backward -> Step |
| **7. Evaluace** | Presnost na testovacich datech |

### Klicove poznatky:
- PyTorch usnadnuje tvorbu neuronovych siti
- DataLoader automatizuje nacitani a michani dat
- Trenovaci smycka je srdcem uceni
- S jednoduchou siti muzeme dosahnout >95% presnosti na MNIST

---

## 13. Vyzva pro vas

Experimentujte s hyperparametry:

In [ ]:
# VYZVA 1: Zmente pocet epoch
print("VYZVA 1: Co se stane pri zmene poctu epoch?")
print("="*50)
print("Zkuste:")
print("  - num_epochs = 1  -> Mene presne?")
print("  - num_epochs = 10 -> Presnejsi?")
print("  - num_epochs = 20 -> Preuceni (overfitting)?")

In [ ]:
# VYZVA 2: Zmente velikost skryte vrstvy
print("VYZVA 2: Co se stane pri zmene hidden_size?")
print("="*50)
print("Zkuste:")
print("  - hidden_size = 32  -> Mensi kapacita")
print("  - hidden_size = 256 -> Vetsi kapacita")
print("  - hidden_size = 512 -> Jeste vetsi")

In [ ]:
# VYZVA 3: Pridejte dalsi skrytou vrstvu
print("VYZVA 3: Pridejte druhou skrytou vrstvu!")
print("="*50)
print()
print("Modifikujte tridu NeuralNet:")
print()
print("class DeeperNet(nn.Module):")
print("    def __init__(self, input_size, hidden1, hidden2, num_classes):")
print("        ...")
print("        self.layer1 = nn.Linear(input_size, hidden1)")
print("        self.layer2 = nn.Linear(hidden1, hidden2)")
print("        self.layer3 = nn.Linear(hidden2, num_classes)")
print()
print("Zlepsi se presnost?")

---

## Dalsi kroky

V pristich kapitolach se naucime:
- CNN (Konvolucni site) pro lepsi rozpoznavani obrazku
- Transfer learning - pouziti predtrenovanych modelu
- Pokrocile techniky regularizace

**Gratulujeme! Prave jste natrénovali svou prvni skutecnou neuronovou sit!**